# Deterministic optimisation (using the fluid model)

Here we optimise over several deterministic objective functions evaluated using the fluid flow model. 

We start by giving an illustrative example of the dynamics of the unsheltered queue $u_t$ given by our fluid model, given inputs for $X_0, \mu_0$ and $\lambda_t, h_t, s_t$ for all $t \in \{1,...,T\}$ which we take directly from the simulation model of Singham et al (2023) (see right hand side of Fig 2 with 70% investment in housing and no decrease in shelter).

In [1]:
import json
import fluid_flow_model as fl
import simulation_model as sm
import deterministic_optimisation as do
import matplotlib.pyplot as plt
import math
import numpy as np
import time

ModuleNotFoundError: No module named 'simpy'

In [ ]:
# Get data
with open('data_singham23.json') as json_file:
    data = json.load(json_file)

# Modelling horizonT_a is decision horizon, T_b is extra modelling horizon - here set to 0
T_a = 5
T_b = 0

# Modeling
model = fl.FluidFlowModel(data, data['solution'], T_a, T_b)
T = [i/365 for i in range((T_a+T_b)*365)]
model.analyse(T)

# Plotting
fig, ax = plt.subplots()
ymax = max(model.h_t + model.sh_t + model.unsh_t)
x = [t/365 for t in range((T_a+T_b)*365)]
ax.plot(x, model.h_t, color = 'green')
ax.plot(x, model.sh_t, color = 'orange')
ax.plot(x, model.unsh_t, color = 'red')
ax.set(xlabel="$d/365$ (Time in years)", ylabel='Number of people',
       title='Number of people housed/sheltered/unsheltered')
ax.legend(["$h_d$", "$s_d$", "$u_d$"], loc="upper left")
ax.grid()
ax.set_ylim(0,ymax*1.05)
plt.show()

####  $\Phi_0$ : linear penalty unsheltered Q and sheltered Q
    # min TimeAvg(E[unsh(t) + c*E[sh(t)]])
    # s.t. total budget constraint
    #      annual minimum build constraint

In [ ]:
import deterministic_optimisation as do
import fluid_flow_model as fl

# Set data for system behaviour
data = {'initial_capacity' : {'housing':4000, 'shelter':1500},
        'initial_demand' : 12000, # initial number of people in system
        'service_mean' : {'housing': 4.0, 'shelter': 0.0}, # in years
        'arrival_rates' : [10.0]*5 + [6.0]*5,
        'budget' : 200000000.0, # in dollars
        'costs_accomm' : {'housing' : 30000.0, 'shelter' : 10000.0}, # cost in dollars per unit
        'baseline_build' : 500} # how many housing units and shelter units we must build at least each year

# Set modeliing options
modeling_options = {'T_a' : 5, # in days: modelling and building
                    'T_b' : 5, # in days: extra modelling following all building
                    'model' : do.FluidModel}

In [ ]:
# Set up problem and solve
problem0 = do.Phi0(data, modeling_options, 'phi0', c=0.3)
tic = time.perf_counter()
problem0.solve('glpk')
toc = time.perf_counter()
print('solved in '+str(toc-tic) + ' seconds.')

####  $\Phi_1$ : quadratic penalty unsheltered Q and sheltered Q
    # min TimeAvg(E[unsh(t)^2] + c*E[sh(t)^2])
    # s.t. total budget constraint
    #      annual minimum build constraint

A quadratic objective function discourages the spending of all surplus budget on one type of accommodation and balances the unshelterd and sheltered queues. We note here that: 

* **Shelter** quickly reduces the unsheltered queue, at the expense of a large **sheltered** population.
* **Housing** gives long-term relief to the system, with an initial large **unsheltered** population.  

In [ ]:
# Set up problem and solve
problem1 = do.Phi1(data, modeling_options, 'phi1', c=0.3)
tic = time.perf_counter()
problem1.solve('ipopt')
toc = time.perf_counter()
print('solved in '+str(toc-tic) + ' seconds.')

####  $\Phi_2$ : Add shape constraints
    # min TimeAvg(E[unsh(t)^2] + c*E[sh(t)^2])
    # s.t. total budget constraint
    #      shape constraint on house building rate: positive and non-decreasing
    #      shape constraint on shelter building rate: positive and non-decreasing then negative. 

We here add shape constraints to reflect the fact that: 

* A five-year budget should be spread out across that horizon rather than being used predominantly in the first year.
* We would like the house building rate to at least stay the same over time, or even ramp up. 
* We would like to encourage a negative shelter build rate towards the end of the horizon to allow the 'conversion' of shelter to housing.

When optimising over the quadratic objective function, we find an optimal solution which has the following features: 

* A constant house building rate over the decision horizon. 
* An initial ramp up of shelter is able to bring the unshelterd queue down.
* Subsequent conversion of shelter to housing makes it affordable to build sufficient housing to obtain a service rate in the long run which is greater than the arrival rate of $2500 \text{ arrivals per year}$.
* The price to pay for conversion is an increased unsheltered population in the short term. 

In [ ]:
# Set up problem and solve
problem2 = do.Phi2(data, modeling_options, 'phi2', c=0.3, shelter_mode = 3)
tic = time.perf_counter()
problem2.solve('ipopt')
toc = time.perf_counter()
print('solved in '+str(toc-tic) + ' seconds.')

In [ ]:
import pandas as pd
df = [['$\Phi_0$']+['Housing']+[str(problem0.problem.data['initial_capacity']['housing'])]+[str(int(round(problem0.h_opt[i],0))) + ' (' + '{:.01%}'.format(((problem0.h_opt[i]-problem0.h_opt[i-1])*problem0.problem.costs_accomm['housing'])/(problem0.problem.budget)) + ')' for i in range(1,len(problem0.h_opt))],
       ['']+['Shelter']+[str(problem0.problem.data['initial_capacity']['shelter'])]+[str(int(round(problem0.s_opt[i],0))) + ' (' + '{:.01%}'.format(((problem0.s_opt[i]-problem0.s_opt[i-1])*problem0.problem.costs_accomm['shelter'])/(problem0.problem.budget)) + ')' for i in range(1,len(problem0.s_opt))],
       ['$\Phi_1$']+['Housing']+[str(problem1.problem.data['initial_capacity']['housing'])]+[str(int(round(problem1.h_opt[i],0))) + ' (' + '{:.01%}'.format(((problem1.h_opt[i]-problem1.h_opt[i-1])*problem1.problem.costs_accomm['housing'])/(problem1.problem.budget)) + ')' for i in range(1,len(problem1.h_opt))],
       ['']+['Shelter']+[str(problem1.problem.data['initial_capacity']['shelter'])]+[str(int(round(problem1.s_opt[i],0))) + ' (' + '{:.01%}'.format(((problem1.s_opt[i]-problem1.s_opt[i-1])*problem1.problem.costs_accomm['shelter'])/(problem1.problem.budget)) + ')' for i in range(1,len(problem1.s_opt))],
       ['$\Phi_2$']+['Housing']+[str(problem2.problem.data['initial_capacity']['housing'])]+[str(int(round(problem2.h_opt[i],0))) + ' (' + '{:.01%}'.format(((problem2.h_opt[i]-problem2.h_opt[i-1])*problem2.problem.costs_accomm['housing'])/(problem2.problem.budget)) + ')' for i in range(1,len(problem2.h_opt))],
       ['']+['Shelter']+[str(problem2.problem.data['initial_capacity']['shelter'])]+[str(int(round(problem2.s_opt[i],0))) + ' (' + '{:.01%}'.format(((problem2.s_opt[i]-problem2.s_opt[i-1])*problem2.problem.costs_accomm['shelter'])/(problem2.problem.budget)) + ')' for i in range(1,len(problem2.s_opt))]]
df = pd.DataFrame(columns=['Problem','Building Type','Initial Capacity', 'Year 1','Year 2','Year 3','Year 4','Year 5'], data=df)
df.style.hide()

#### Comparing the unsheltered population for the 3 solutions

In [ ]:
# Modeling
model0 = fl.FluidFlowModel(data, {'housing':problem0.h_opt,'shelter':problem0.s_opt}, modeling_options['T_a'], modeling_options['T_b'])
model1 = fl.FluidFlowModel(data, {'housing':problem1.h_opt,'shelter':problem1.s_opt}, modeling_options['T_a'], modeling_options['T_b'])
model2 = fl.FluidFlowModel(data, {'housing':problem2.h_opt,'shelter':problem2.s_opt}, modeling_options['T_a'], modeling_options['T_b'])

T = [i/365 for i in range((modeling_options['T_a']+modeling_options['T_b'])*365)]

model0.analyse(T)
model1.analyse(T)
model2.analyse(T)

# Plotting
fig, ax = plt.subplots()
ymax = max(model0.unsh_t + model1.unsh_t + model2.unsh_t)
x = [t/365 for t in range((modeling_options['T_a']+modeling_options['T_b'])*365)]
ax.plot(x, model0.unsh_t, color = 'pink')
ax.plot(x, model1.unsh_t, color = 'red')
ax.plot(x, model2.unsh_t, color = 'darkred')
ax.set(xlabel="$d/365$ (Time in years)", ylabel='Number of people',
       title='Number of people unsheltered')
ax.legend(["Optimal solution to $\Phi_0$", "Optimal solution to $\Phi_1$", "Optimal solution to $\Phi_2$"], loc="lower right")
ax.grid()
ax.set_ylim(0,ymax*1.05)
plt.show()

#### Checking the costs of optimal solutions

In [ ]:
def cost(problem):
    costs = 0
    for t in range(problem.problem.horizon_decision):
        costs += (problem.h_opt[t+1]-problem.h_opt[t]) * problem.problem.costs_accomm['housing']
        costs += (problem.s_opt[t+1]-problem.s_opt[t]) * problem.problem.costs_accomm['shelter']
    return costs

print(cost(problem0))
print(cost(problem1))
print(cost(problem2))

#### Exploring different weights for penalty on shelter

In [ ]:
# Set up problem and solve
problems = []
models = []
weights = [0.2,0.4,0.6,0.8] 
for w in weights:
    problems.append(do.Phi2(data, modeling_options, 'phi2', c=w, shelter_mode = 3))
    problems[-1].solve('ipopt', print = False)
    models.append(problems[-1].problem.selected_model(problems[-1].problem.data, 
                                                      {'housing' : problems[-1].h_opt, 'shelter' : problems[-1].s_opt}, 
                                                      problems[-1].problem.horizon_decision, 
                                                      problems[-1].problem.horizon_extra_model))
    models[-1].analyse(problems[-1].problem.horizon_model)

In [ ]:
fig, axs = plt.subplots(1, 4,  figsize=(10, 3))
ymax = 0
for i in range(len(models)):
    ymax = max(ymax, max(models[i].model.h_t + models[i].model.sh_t + models[i].model.unsh_t))
    x = [j/365 for j in range(problems[i].problem.horizon_model*365)]
    axs[i].plot(x, models[i].model.h_t, color = 'green')
    axs[i].plot(x, models[i].model.sh_t, color = 'orange')
    axs[i].plot(x, models[i].model.unsh_t, color = 'red')
    axs[i].set(title = 'w = ' + str(weights[i]))
axs[i].legend(["$h_d$", "$s_d$", "$u_d$"], bbox_to_anchor=(1.1, 0.7))

# formatting
for ax in axs.flat:
    ax.set(xlabel='t (yrs)', ylabel='Number of people')
    ax.grid()
    ax.set_ylim(0, ymax*1.05)
    ax.label_outer()
plt.show()

In [ ]:
# DES
        'service_mean' : {'housing': 4.0, 'shelter': 0.0}, # in years
        'arrival_rates' : [10.0]*5 + [6.0]*5,
        'budget' : 200000000.0, # in dollars
        'costs_accomm' : {'housing' : 30000.0, 'shelter' : 10000.0}, # cost in dollars per unit
        'baseline_build' : 500} # how many housing units and shelter units we must build at least each year

# Set modeliing options
modeling_options = {'T_a' : 5, # in days: modelling and building
                    'T_b' : 5, # in days: extra modelling following all building
                    'model' : do.FluidModel}

simulation_data = {"T_a":5,
 "T_b":5,
 "initial_capacity": {"housing": 40, "shelter": 15},
 "initial_demand": 120,
 "service_mean": {"housing": 4.0, "shelter": 0.25},
 "service_dist_triangle": {"low":0,"mid":6,"high":8},
 "arrival_rates" : [36.5]*5 + [(6/10)*36.5]*5
 "solution": {'housing':problem2.h_opt,'shelter':problem2.s_opt},
 "max_in_system":500,
 "delta_t":1,
 "simulation_reps":100,
 "time_btwn_building":0.0027397260273972603,
 "simulation_build_time":0,
 "reentry_rate":0.17,
 "seed":12345}

s = sm.SimulationModel(simulation_data, simulation_data["solution"])
s.analyse()